# Xfinity Offer Scraping
#### By Julia Geller


In [42]:
import requests
import regex as re
import json
import numpy as np

In [188]:
def get_offer(house_number: str, street_name: str, street_type:str, city: str, state: str, zip_code: str, lat: float, long: float):
    """
    Gets the response from https://www.xfinity.com/learn/landing/active-address for searching for 
    an offer at the given address in the United States.
    Parameters:
        house_number - str house number for the address
        street_name - str street name (i.e. Huntington)
        street_type - str type of street (i.e. Avenue, Boulevard, etc.)
        city - str name of city
        state - str state abberviation (i.e. MA)
        zip_code - str zip code
        lat - float latitude of the address
        long - float longitude for the address
    Return:
    json offer found for the given address
    """
    # https://www.xfinity.com/learn/landing/active-address
    cookies = {
    'PSC': 'UCID=1c421100-42fa-4bd7-8ac4-6f4285dc29de&CTY={0}&ST={1}&Z={2}&EX=False&REC=N&RC.MKT=5091&RC.RLOC=232548872'.format(city.upper(), state.upper(), zip_code),
    '__privaci_cookie_consents': '{"consents":{"77":1,"78":1,"79":1,"80":1},"location":"'+ state.upper()+'#US","lang":"en","gpcInBrowserOnConsent":false,"gpcStatusInPortalOnConsent":false,"status":"record-consent-success","implicit_consent":true}',
    'SC': 'RC.USID=f4c78f88-d973-4999-9d80-38cf98646d69&VA=71681&RC.IFL=Y&RC.LOC=232548872&L1ID=11599&L2ID=5051&L3ID=6132&L4ID=3846&RC.IPL=N&RC.SP=d8400ca2-eb37-4f2d-8b0d-0d4651452e71&Z=[{0}]&CTY={1}&ST={2}&RC.MKT=5091&RC.BIL=CSG&RC.SYS=8773&RC.PRIN=1000&RC.AGT=800&RC.CORP=0&RC.CRPN=0&RC.FRCH=0&RC.FRCHN=800&RC.HK=38285288EA8EE203FA316A90971A6848'.format(zip_code, city.upper(), state.upper()),
    }
    
    response = requests.post('https://digital.xfinity.com/offers/api/offerpackaging', cookies=cookies, headers={}, json={})
    
    return response.json()


In [ ]:
def process_offer(offer: json):
    """
    Processes the offer returned from from https://www.xfinity.com/learn/landing/active-address for a specific address.
    Parameters:
        offer - json response from searching for the offer using get_offer()
    Return:
    plan_name - str name of the cheapest internet plan
    price - str price in dollars of the cheapest internet plan
    speed_down - str download speed for the cheapest internet plan
    units_down = str units for the download speed for the cheapest internet plan
    speed_up - str upload speed for the cheapest internet plan
    units_up - str units for the upload speed for the cheapest internet plan
    """
    content = offer['CONTENT']

    # get all plans and price for plan
    plan_price = [(content[key]['Name'],content[key]['EdpPrice']) for key in content.keys() if 'InternetPlan' in content[key].keys()]
    # filter out plans that are not full internet service, manually verified
    plan_price = [pp for pp in plan_price if pp[0] not in ['Internet Essentials', 'Internet Essentials Plus']]
    cheapest_plan = plan_price[np.argmin([eval(pp[1]) for pp in plan_price])]

    plan_name = cheapest_plan[0]
    price = cheapest_plan[1]

    # get details for cheapest plan
    plan_details = [content[key] for key in content.keys() if content[key]['Name']==plan_name][0]

    # get upload speed and units
    upload_info = plan_details['UploadSpeed'].split(' ')
    speed_up= upload_info[len(upload_info)-2]
    units_up= upload_info[len(upload_info)-1]

    # get upload speed and units
    download_info = plan_details['DownloadSpeed'].split(' ')
    speed_down = download_info[len(download_info)-2]
    units_down = download_info[len(download_info)-1]

    return plan_name, price,speed_down, units_down, speed_up, units_up